In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-02 19:26:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.07MB/s    in 0.2s    

2021-07-02 19:26:52 (5.07 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [5]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [6]:
# Filter total votes to show only over 20 total votes
over20_vote_df = vine_df.filter("total_votes>20")

In [7]:
over20_vote_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R3EM8C9CKA6GPK|          2|           41|        115|   N|                N|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|     

In [8]:
# Filter the vine table to show the helpful votes with 50% and more portion of total votes. 
filtered_vine = over20_vote_df.filter((over20_vote_df.helpful_votes/over20_vote_df.total_votes) > 0.5)

In [9]:
filtered_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [10]:
# Create separate dataframe for paid and unpaid vine program
paid_vine = filtered_vine.filter("vine == 'Y'")

In [11]:
paid_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|
| RGXLFTR31J561|          5|           29|         34|   Y|                N|
|R2ZS3WCJYUDQH0|          3|          101|        115|   Y|                N|
|R31771VBPRDVJS|          4|           75|         83|   Y|                N|
| R8O8KM73GO190|          5|           21|         28|   Y|                N|
| RPU1GURA1NOT0|          5|           33|         35|   Y|     

In [12]:
# Create separate dataframe for paid and unpaid vine program
unpaid_vine = filtered_vine.filter("vine == 'N'")

In [13]:
unpaid_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [14]:
# This is extra report to just have a summary of total overview of each star rating for paid vine program.
paid_vine_summary = paid_vine.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)","Total Review(Paid)")

In [15]:
from pyspark.sql.functions import col
paid_vine_summary = paid_vine_summary.orderBy(paid_vine_summary["star_rating"].desc())
paid_vine_summary.show()

+-----------+------------------+
|star_rating|Total Review(Paid)|
+-----------+------------------+
|          5|               246|
|          4|               198|
|          3|                99|
|          2|                20|
|          1|                16|
+-----------+------------------+



In [16]:
# This extra report to just have a summary of total overview of each star rating for unpaid vine program.
unpaid_vine_summary = unpaid_vine.groupby("star_rating").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)","Total Review(Unpaid)")

In [17]:
unpaid_vine_summary = unpaid_vine_summary.orderBy(unpaid_vine_summary["star_rating"].desc())
unpaid_vine_summary.show()

+-----------+--------------------+
|star_rating|Total Review(Unpaid)|
+-----------+--------------------+
|          5|               23816|
|          4|                9581|
|          3|                4726|
|          2|                3018|
|          1|                6380|
+-----------+--------------------+



In [18]:
from pyspark.sql.functions import sum,avg,count

In [19]:
# Calculate total review of paid vine program
total_paid_review = paid_vine_summary.select(sum("Total Review(Paid)")).show()

+-----------------------+
|sum(Total Review(Paid))|
+-----------------------+
|                    579|
+-----------------------+



In [20]:
# Calculate total review of unpaid vine program
total_unpaid_review = unpaid_vine_summary.select(sum("Total Review(Unpaid)")).show()

+-------------------------+
|sum(Total Review(Unpaid))|
+-------------------------+
|                    47521|
+-------------------------+



In [21]:
# Second method to calculate total review of paid and unpaid vine program
paid_vine.select(count("star_rating")).withColumnRenamed("count(star_rating)","Total Review Paid Program").show()
unpaid_vine.select(count("star_rating")).withColumnRenamed("count(star_rating)","Total Review Unpaid Program").show()

+-------------------------+
|Total Review Paid Program|
+-------------------------+
|                      579|
+-------------------------+

+---------------------------+
|Total Review Unpaid Program|
+---------------------------+
|                      47521|
+---------------------------+



In [22]:
# Calculate total number of 5 star reviews for paid vine progtram
total_5_star_paid = paid_vine.filter("star_rating == 5").select(count("star_rating")).withColumnRenamed("count(star_rating)","Total_5_Star_Reviews_Paid")
total_5_star_paid.show()

+-------------------------+
|Total_5_Star_Reviews_Paid|
+-------------------------+
|                      246|
+-------------------------+



In [23]:
# Calculate total number of 5 star reviews for unpaid vine progtram
total_5_star_unpaid = unpaid_vine.filter("star_rating == 5").select(count("star_rating")).withColumnRenamed("count(star_rating)","Total 5 Star Reviews Unpaid").show()

+---------------------------+
|Total 5 Star Reviews Unpaid|
+---------------------------+
|                      23816|
+---------------------------+



In [24]:
# Calculate percentage of 5-star review for paid and unpaid program
paid_5_star_percent = (246 / 579) * 100
unpaid_5_star_percent = (23816 / 47521) * 100

In [25]:
summary_df = spark.createDataFrame([
                                   ("Paid", paid_5_star_percent),
                                   ("Unpaid", unpaid_5_star_percent)                                   
], ["Vine Program","5 Star Review Percentage"])
summary_df.show()

+------------+------------------------+
|Vine Program|5 Star Review Percentage|
+------------+------------------------+
|        Paid|      42.487046632124354|
|      Unpaid|      50.116790471580984|
+------------+------------------------+

